In [2]:
from sqlalchemy import create_engine
import pandas as pd
import os
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')
table_name = 'test_table'
connection_string = f"mysql+mysqlconnector://${MYSQL_USER}:${MYSQL_PASSWORD}@db:3306/${MYSQL_DATABASE}"
engine = create_engine(connection_string)

# Example DataFrame to save
data = pd.DataFrame({'column1': [1, 2, 3], 'column2': ['A', 'B', 'C']})

# Save the DataFrame to the MySQL table
data.to_sql(table_name, engine, index=False, if_exists='replace')

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:db

In [2]:
import os

from minio import Minio
from minio.error import InvalidResponseError

accessID = os.environ.get('AWS_ACCESS_KEY_ID')
accessSecret =  os.environ.get('AWS_SECRET_ACCESS_KEY')
minioUrl =  os.environ.get('MLFLOW_S3_ENDPOINT_URL')
minioUrl = 'http://s3:9000'
bucketName =  'ree'

if accessID == None:
    print('[!] AWS_ACCESS_KEY_ID environment variable is empty! run \'source .env\' to load it from the .env file')
    exit(1)

if accessSecret == None:
    print('[!] AWS_SECRET_ACCESS_KEY environment variable is empty! run \'source .env\' to load it from the .env file')
    exit(1)

if minioUrl == None:
    print('[!] MLFLOW_S3_ENDPOINT_URL environment variable is empty! run \'source .env\' to load it from the .env file')
    exit(1)

    
if bucketName == None:
    print('[!] AWS_BUCKET_NAME environment variable is empty! run \'source .env\' to load it from the .env file')
    exit(1)

minioUrlHostWithPort = minioUrl.split('//')[1]
print('[*] minio url: ',minioUrlHostWithPort)

s3Client = Minio(
    minioUrlHostWithPort,
    access_key=accessID,
    secret_key=accessSecret,
    secure=False
)

s3Client.make_bucket(bucketName)

[*] minio url:  s3:9000


In [1]:
import mlflow
# set url to your mlflow server
# mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("Default")
with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("foo", 1)
    mlflow.log_metric("foo", 2)
    mlflow.log_metric("foo", 3)
    mlflow.log_artifact("00-skeleton.ipynb")


In [2]:
!echo  $MLFLOW_TRACKING_URI


http://mlflow:5000


In [1]:
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import config
import os
# ignore warnings
from merlin.core.dispatch import get_lib
from merlin.schema.tags import Tags

import time
import merlin.models.tf as mm
# from dask.distributed import Client, wait
# from dask_cuda import LocalCUDACluster
# from dask.utils import parse_bytes

from merlin.schema.tags import Tags
from nvtabular.ops import *

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-08-05 13:40:15.589748: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-05 13:40:15.910564: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-08-05 13:40:17.828875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-05 13:40:17.829232: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-05 13:40:17.829330: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-08-05 13:40:18.377397: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-05 13:40:18.377519: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-05 13:40:18.377567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [2]:
train = pd.read_parquet(os.path.join(config.data_raw_dir,'transactions.parquet'))
customers = pd.read_parquet(os.path.join(config.data_raw_dir,'customers.parquet'))

encoder = LabelEncoder()
merged_ids = pd.concat([train['customer_id'], customers['customer_id']])
encoder.fit(merged_ids)
train['customer_id'] = encoder.transform(train['customer_id'])
customers['customer_id'] = encoder.transform(customers['customer_id'])
customers.to_parquet(os.path.join(config.data_processed_dir,'customers_enc.parquet'))
del customers




In [3]:
# convert  t_dat column to datetime
train['t_dat'] = pd.to_datetime(train['t_dat'])
train['week'] = 104 -(train['t_dat'].max() - train['t_dat']).dt.days // 7
start = 60
end = 90
valid = train[train.week>end]
train = train[(train.week>start) & (train.week<=end)]
valid = valid[valid['customer_id'].isin(train['customer_id'])]


In [4]:
print('train shape', train.shape)
print('valid shape', valid.shape)

train shape (8231156, 6)
valid shape (3706105, 6)


In [5]:
# see how many 'customer_id' are common between the two datasets train and valid and also items 'article_id'
common = set(train['customer_id']).intersection(set(valid['customer_id']))
common_items = set(train['article_id']).intersection(set(valid['article_id']))
print('Number of common customers between train and valid: ', len(common))
print('Number of common items between train and valid: ', len(common_items))
print('Number of customers in train: ', len(train['customer_id'].unique()))
print('Number of customers in valid: ', len(valid['customer_id'].unique()))
print('Number of items in train: ', len(train['article_id'].unique()))
print('Number of items in valid: ', len(valid['article_id'].unique()))
print('Number of customers in valid but not in train: ', len(set(valid['customer_id']) - set(train['customer_id'])))
print('Number of items in valid but not in train: ', len(set(valid['article_id']) - set(train['article_id'])))
print('train shape: ', train.shape)
print('valid shape: ', valid.shape)

Number of common customers between train and valid:  408937
Number of common items between train and valid:  31933
Number of customers in train:  780121
Number of customers in valid:  408937
Number of items in train:  55238
Number of items in valid:  42459
Number of customers in valid but not in train:  0
Number of items in valid but not in train:  10526
train shape:  (8231156, 6)
valid shape:  (3706105, 6)


In [10]:
save_dir = os.path.join(config.data_processed_dir,'train', 'transactions.parquet')
train.to_parquet(save_dir)

save_dir = os.path.join(config.data_processed_dir,'valid', 'transactions.parquet')
valid.to_parquet(save_dir)
del train, valid

In [7]:
train = nvt.Dataset(train)

In [11]:
train = nvt.Dataset(os.path.join(config.data_processed_dir,'train', 'transactions.parquet'),engine="parquet",  part_mem_fraction=0.1)
valid = nvt.Dataset(os.path.join(config.data_processed_dir,'valid', 'transactions.parquet'),engine="parquet", part_mem_fraction=0.1)

In [14]:
train.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week
0,2019-11-20,155,828652001,0.016932,1,61
1,2019-11-20,330,408647009,0.016932,2,61
2,2019-11-20,330,572797002,0.010153,2,61
3,2019-11-20,330,634426008,0.042356,2,61
4,2019-11-20,330,753421001,0.067780,2,61


In [15]:
article_id = ["article_id"] >> Categorify() >> TagAsItemID()
customer_id = ["customer_id"] >> Categorify() >> TagAsUserID()
price = ["price"] >> FillMissing(fill_val=0) >> Normalize() >> TagAsItemFeatures()
len_session = ["price"] >> FillMissing(fill_val=0) >> Normalize() >> TagAsUserFeatures()


features = article_id + customer_id + price + len_session

wf = nvt.Workflow(features)
wf.fit_transform(train).to_parquet(os.path.join(config.data_processed_dir, 'train_processed.parquet'))
wf.transform(valid).to_parquet(os.path.join(config.data_processed_dir, 'validation_processed.parquet'))

wf.save(os.path.join(config.data_processed_dir, 'workflow'))

In [16]:
train = Dataset(os.path.join(config.data_processed_dir, 'train_processed.parquet'))
valid = Dataset(os.path.join(config.data_processed_dir, 'validation_processed.parquet'))
schema = train.schema

In [17]:
schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,article_id,"(Tags.ID, Tags.CATEGORICAL, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.article_id.parquet,55241.0,512.0,0.0,55240.0,article_id
1,customer_id,"(Tags.USER, Tags.CATEGORICAL, Tags.ID)","DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.customer_id.parquet,780124.0,512.0,0.0,780123.0,customer_id
2,price,"(Tags.CONTINUOUS, Tags.USER, Tags.ITEM)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

In [19]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([512, 256, 128, 64], no_activation_last_layer=True),
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
)

In [21]:
model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(30), mm.NDCGAt(30)])
model.fit(train, validation_data=valid, batch_size=4000, epochs=15)

Epoch 1/15


2023-08-03 15:57:15.260244: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2057/2058 [============================>.] - ETA: 0s - loss: 5.4461 - recall_at_30: 0.3217 - ndcg_at_30: 0.1081 - regularization_loss: 0.0000e+00 - loss_batch: 5.4461

2023-08-03 15:58:34.897051: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
The sampler InBatchSampler returned no samples for this batch.


2058/2058 [==============================] - 92s 44ms/step - loss: 5.4460 - recall_at_30: 0.3217 - ndcg_at_30: 0.1082 - regularization_loss: 0.0000e+00 - loss_batch: 5.4458 - val_loss: 5.3480 - val_recall_at_30: 0.3605 - val_ndcg_at_30: 0.1344 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.1751
Epoch 2/15
2058/2058 [==============================] - 90s 44ms/step - loss: 5.2267 - recall_at_30: 0.3987 - ndcg_at_30: 0.1390 - regularization_loss: 0.0000e+00 - loss_batch: 5.2265 - val_loss: 5.1810 - val_recall_at_30: 0.3442 - val_ndcg_at_30: 0.1284 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.1086
Epoch 3/15
2058/2058 [==============================] - 90s 44ms/step - loss: 4.8108 - recall_at_30: 0.4268 - ndcg_at_30: 0.1526 - regularization_loss: 0.0000e+00 - loss_batch: 4.8106 - val_loss: 4.9178 - val_recall_at_30: 0.3810 - val_ndcg_at_30: 0.1519 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 3.8806
Epoch 4/15
2058/2058 [==============================] - 9

In [22]:
model.evaluate(valid, batch_size=1000, return_dict=True)

  35/3707 [..............................] - ETA: 11s - loss: 3.7486 - recall_at_30: 0.7791 - ndcg_at_30: 0.3592 - regularization_loss: 0.0000e+00 - loss_batch: 3.7486

2023-08-03 16:21:29.298362: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3707/3707 [==============================] - 10s 3ms/step - loss: 4.0384 - recall_at_30: 0.7330 - ndcg_at_30: 0.3295 - regularization_loss: 0.0000e+00 - loss_batch: 4.0371


{'loss': 4.038394451141357,
 'recall_at_30': 0.7511730194091797,
 'ndcg_at_30': 0.3364556133747101,
 'regularization_loss': 0.0,
 'loss_batch': 1.5547443628311157}

In [43]:
model.evaluate(valid, batch_size=1000, return_dict=True)

  34/3089 [..............................] - ETA: 9s - loss: 7.0239 - recall_at_30: 0.0353 - ndcg_at_30: 0.0112 - regularization_loss: 0.0000e+00 - loss_batch: 7.0239

2023-08-02 16:03:29.295270: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3089/3089 [==============================] - 9s 3ms/step - loss: 7.0094 - recall_at_30: 0.0374 - ndcg_at_30: 0.0119 - regularization_loss: 0.0000e+00 - loss_batch: 7.0092


{'loss': 7.009433269500732,
 'recall_at_30': 0.03781740367412567,
 'ndcg_at_30': 0.012081457301974297,
 'regularization_loss': 0.0,
 'loss_batch': 6.611358165740967}